In [2]:
import os

os.environ["GROQ_API_KEY"] = "YOUR_API_KEY"

In [18]:
import csv

csv_file_path = "Three_EPS_Req.csv"

def load_csv(file_path):
    requirements = []
    with open(file_path, 'r') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            requirements.append(row)
    return requirements
requirements_doc = load_csv(csv_file_path)

In [123]:
import re
text = requirements_doc[2]['Primary Text']

cleaned_string = re.sub(r'[^a-zA-Z0-9]]', '', text)

print(cleaned_string)

The ECU shall have a maximum power consumption of 50W during peak load conditions (e.g., full steering assist torque at low vehicle speeds).

During normal operation, power consumption shall not exceed 20W, optimizing energy usage under typical driving conditions.


In [132]:
## LANCEDB CODE

from agno.knowledge.pdf import PDFKnowledgeBase, PDFReader
from agno.knowledge.csv import CSVKnowledgeBase
from agno.vectordb.lancedb import LanceDb
from agno.vectordb.search import SearchType
from agno.embedder.sentence_transformer import SentenceTransformerEmbedder
from agno.document.chunking.recursive import RecursiveChunking
from agno.knowledge.combined import CombinedKnowledgeBase

pdf_knowledege_base = PDFKnowledgeBase(
    path="EPS_Info.pdf",
    # Table name: ai.pdf_documents
    vector_db=LanceDb(table_name="Information",
                      uri="vectordv_lancedb/lancedb_pdf",
                    embedder = SentenceTransformerEmbedder("all-MiniLM-L6-v2"),
                    search_type=SearchType.hybrid),
    num_documents = 5,
    chunking_strategy=RecursiveChunking(chunk_size = 500),
)
# pdf_knowledege_base.load()

csv_knowledge_base = CSVKnowledgeBase(
    path="Three_EPS_Req.csv",
    # Table name: ai.csv_documents
     vector_db=LanceDb(table_name="Requirements",
                      uri="vectordv_lancedb/lancedb_csv",
                    embedder = SentenceTransformerEmbedder("all-MiniLM-L6-v2")
                ),
    num_documents = 5,
    chunking_strategy=RecursiveChunking(chunk_size = 500)
)
# csv_knowledge_base.load()

knowledge_base = CombinedKnowledgeBase(
    sources=[
        pdf_knowledege_base,
        csv_knowledge_base,
    ],
   vector_db=LanceDb(table_name="Combined_documents",
                      uri="vectordv_lancedb/combined_db",
                    embedder = SentenceTransformerEmbedder("all-MiniLM-L6-v2")
                ),
    num_documents = 6,
    chunking_strategy=RecursiveChunking(chunk_size = 500)
)

knowledge_base.load()

INFO     Creating collection

INFO     Loading knowledge base

INFO     Reading: EPS_Info

INFO     Added 0 documents to knowledge base

INFO     Reading: Three_EPS_Req.csv

INFO     Added 0 documents to knowledge base

In [3]:
task="requirment id: 245114"

In [33]:
description = """ You are an AI assisant who will help understanding the requirements of automotive Electronic Circuit Units (ECU). 
For given system requirement, perform the below task.
- Understand the requirement completely based on provided knowledge base and Explain your understanding and thoughts. 
- If you need even more inputs to understand the requirement, please ask to user as clarifications required,
- Based on your understanding generate test sceanrios to fully validate the requirement"""

In [75]:
from textwrap import dedent

output_format = """Give the response in below format
<requirement_objective>
{Objective of Requirement with 10 word}
</requirement_objective>

<context>
{Your Understanding of Requirement}
</context>

<test_sceanrios>
{Test Scenarios for validating the requirement}
<test_sceanrios>

</clarifications>
{Your Questions / Required clarification points, if any}
</clarifications>
"""

In [6]:
# instructions = """Analyze Input Requirements: Carefully read and understand the provided system requirements related to automotive ECUs, focusing on functional behavior, hardware-software interactions, safety constraints, and communication protocols.
# Identify Ambiguities: Detect any gaps, ambiguities, or inconsistencies in the requirements that may lead to misinterpretation during development or validation.

# Ask Clarifying Questions: Formulate clear, concise, and technically relevant questions to clarify any vague points. 

# Maintain Technical Depth: Ensure your questions reflect a deep understanding of ECU architectures and provided knowledge base.

# Prioritize Clarity and Completeness: Your goal is to ensure the requirements are fully understood and leave no room for assumptions in the implementation phase.
# Example:
#     Ambiguity: "The ECU should respond quickly to driver inputs."
#     Clarifying Question: "What is the specific response time threshold (in milliseconds) expected for driver inputs under normal and fault conditions?"

# Note: Clarification provided understand the requirement again based on clarification points.
# """

In [43]:
instructions = [
    "If find any ambiguity in understanding requirement, please ask for clarifications. Don't do any assumptions, you should generate content based on knowledge base only",
    "You should generate test scenarios that will fully verify the provided system requirement. If you feel you need more information in order to make sure the requirement is fully validated, ask user to give more inputs",
    "Once user provides you the clarifications, use those points to understand requirements and to generate test scenarios.",
    "if requirements are clear or no additional information required for you, print 'None' in clarification section of expected output"
]

In [38]:
from agno.agent import Agent, AgentMemory
from agno.memory.db.sqlite import SqliteMemoryDb
from agno.memory.classifier import MemoryClassifier
from agno.memory.summarizer import MemorySummarizer
from agno.models.groq import Groq

# Define persistent memory for chat history
memory=AgentMemory(
        db=SqliteMemoryDb(
            table_name="agent_memory",
            db_file="tmp/agent_memory.db"),
        # Create and store personalized memories for this user
        create_user_memories=True,
        # Update memories for the user after each run
        update_user_memories_after_run=True,
        # Create and store session summaries
        create_session_summary=True,
        # Update session summaries after each run
        update_session_summary_after_run=True,
        classifier=MemoryClassifier(model=Groq(id="llama-3.3-70b-versatile")),
        summarizer=MemorySummarizer(model=Groq(id="llama-3.3-70b-versatile")),
    )     

In [129]:
from agno.agent import Agent, RunResponse
from agno.models.groq import Groq
from agno.storage.agent.sqlite import SqliteAgentStorage
import typer

# def create_agent(user: str = "user"):
#     """
#         Function which contains the agent information and definition.
#         Returns: The Agent
#     """
# session_id: Optional[str] = None

# Ask if user wants to start new session or continue existing one
# new = typer.confirm("Do you want to start a new session?")

# if not new:
#     existing_sessions = agent_storage.get_all_session_ids(user)
#     if len(existing_sessions) > 0:
#         session_id = existing_sessions[0]

agent = Agent(
    knowledge=knowledge_base,
    # session_id=session_id,
    search_knowledge=True,
    add_references=True,
    model=Groq(id="deepseek-r1-distill-llama-70b", temperature=0.5),
    description = description,
    markdown=True,
    instructions=instructions,
    expected_output=output_format,
    structured_outputs=True,
    retries = 2,
    # update_knowledge=True,
    # memory=memory,
    # Store agent sessions in a database, that persists between runs
    # storage=SqliteAgentStorage(
    #     table_name="agent_sessions", db_file="tmp/agent_storage.db"
    # ),
    add_history_to_messages=True, # adds the chat history to the messages sent to the Model.
    num_history_responses=20   # Number of historical responses to add to the messages.
)

    # if session_id is None:
    #     session_id = agent.session_id
    #     if session_id is not None:
    #         print(f"Started Session: {session_id}\n")
    #     else:
    #         print("Started Session\n")
    # else:
    #     print(f"Continuing Session: {session_id}\n")
    
    # return agent

# agent.knowledge.load(recreate=False)

In [98]:
from rich.pretty import pprint

# agent = create_agent()
# pprint(agent.run(task))

In [116]:
import re

def parse_agent_output(output: str, tag: str):
    match = re.search(f'<{tag}>\s*(.*?)\s*</{tag}>', output, re.DOTALL)
    print(match)
    if match:
        clarifications = match.group(1).strip()
        return clarifications

In [117]:
print(parse_agent_output("<clarifications>- Specific torque values and vehicle speeds for testing.- Duration of each test for accurate measurement.</clarifications>", "clarifications"))

<re.Match object; span=(0, 138), match='<clarifications>- Specific torque values and vehi>
- Specific torque values and vehicle speeds for testing.- Duration of each test for accurate measurement.


In [134]:
from rich.console import Console
from rich.prompt import Prompt
from agno.agent import Agent
from rich.pretty import pprint
import time

console = Console()

# def validate_agent_output(response: str) -> str | None:
#     """Providing clarification to agent"""

#     clarification = Prompt.ask("Please clarify the questions")
#     console.print(f"[green]Modified response accepted:[/]\n{clarification}")

    # if detect_ambiguity(response):
    #     console.print("[red]Ambiguity detected in agent's response. Stopping execution.[/]")
    #     raise StopAgentRun(
    #         "######### Clarification required ######",
    #         agent_message="Stopping execution since clarification required"
    #     )
        
    
    # return clarification  # Use modified response

def pre_hook_agent_output(requirement, output: str):
    """Function to validate agent output"""
    
    message = Prompt.ask("Satiesfied with agent response ?", choices=["y", "n"], default="y").strip().lower()
    if message != "y":
        console.print("[red]OKAY, User not satisfied\n[/]")
        rerun_agent = Prompt.ask("What to re-run the Agent for better results ? ?", choices=["y", "n"], default="n").strip().lower()
        if rerun_agent == "y":
            while True:
                final_response = agent.run(f"Run the agent again and provide better results for {requirement}")
                print(final_response.content.strip())

                console.print("[blue]Waiting for 55 sec")
                time.sleep(55)
                
                rerun_agent_2 = Prompt.ask("What to re-run the Agent for better results ? ?", choices=["y", "n"], default="n").strip().lower()

                if rerun_agent_2 == 'n':
                    break
                
        else:
            return False 
    return True

def ask_agent(question: str, agent: Agent):
    """Requirement Understanding and clarification"""
    
    console.print(f"\n[bold blue]Agent is generating a response for[/] {question}")
    response = agent.run(question)  # Capture raw agent output

    #pprint(response)
    
    response_content = response.content.strip()
    
    # req_objective = extract_xml(response_content, "requirement_objective")
    # context = extract_xml(response_content, "context")
    # test_scenarios = extract_xml(response_content, "test_scenarios")
    clarifications = parse_agent_output(response_content, "clarifications")
    console.print(f"\n[bold blue]Agent response preview[/]\n {response_content}\n\n")

    console.print(f"Clarification:  {clarifications}")

    final_response = response_content
    
    while True:
        temp = parse_agent_output(str(final_response), "clarifications")
        
        if str(temp) != "None":
            console.print("[bold red]Agent Need Clarification")
            inputs = Prompt.ask("Please clarify the questions")
            
            #console.print(f"[green]Modified response accepted:[/]\n{clarification}")
            # validated_response = validate_agent_output(response_content)

            console.print("[blue]Waiting for 40 sec")
            time.sleep(40)

            console.print(f"\n[bold blue]Agent generating the response again")
    
            final_response = agent.run(f"Here are the inputs from user: {inputs}. Now use these points, and generate the test scenarios again")
            final_response = final_response.content.strip()
            
            console.print(f"\n[blue]Intermediate output:[/]\n {final_response}")
        
        elif pre_hook_agent_output(question, response_content):
            console.print("[bold yellow]User satisfied.[/]")
            break
        else:
            console.print("[bold red]Agent wouldn't generated any response.[/]")
            break
    
    # if final_response:
    #     console.print("\n[bold green]Final Response:[/]")
    #     console.print(final_response.content.strip())
    # else:
    #     console.print("[yellow]Agent output was skipped.[/]")

In [135]:
ask_agent(task, agent)

Agent is generating a response for requirment id: 245114

<re.Match object; span=(2507, 3081), match='<clarifications>\n- Clarify if the ECU is designe>


Agent response preview
 <requirement_objective>
ECU Operation and Power Efficiency Across Voltage Ranges
</requirement_objective>

<context>
The requirement specifies that the Steering ECU must operate reliably within both 12V and 48V vehicle electrical 
systems, ensuring efficient power consumption under various load conditions. It also highlights the ECU's role in 
responding to steering torque input by generating appropriate assistive torque to aid vehicle maneuvering.
</context>

<test_sceanrios>
1. **Voltage Range Test**:
   - Operate the ECU at 12V and 48V nominal voltages.
   - Verify reliable operation and consistent assistive torque generation under both voltages.

2. **Voltage Fluctuation Test**:
   - Simulate voltage fluctuations within the specified ranges (e.g., 9V to 15V for 12V system, appropriate 
fluctuations for 48V).
   - Ensure the ECU maintains operation and adjusts power consumption efficiently.

3. **Load Variation Test**:
   - Apply varying electrical loads to simulate different driving conditions.
   - Measure power consumption to confirm efficiency under high and low loads.

4. **Torque Response Test**:
   - Apply varying steering torque inputs to the ECU.
   - Measure the ECU's assistive torque output to ensure correct and timely responses.

5. **Power Consumption Efficiency Test**:
   - Define and measure the metrics for efficient power consumption (e.g., current draw, power draw).
   - Test under different load conditions to ensure efficiency is maintained.

6. **Voltage Transition Test**:
   - Switch the ECU between 12V and 48V systems.
   - Monitor for smooth operation and consistent power consumption during transitions.

7. **Fault Condition Test**:
   - Simulate over-voltage and under-voltage conditions beyond the specified ranges.
   - Verify the ECU's response, including any protective mechanisms or alerts.

8. **Environmental Test**:
   - Test the ECU's operation and power efficiency across a range of temperatures (e.g., -40°C to 125°C).
   - Ensure reliable performance and consistent power consumption in various environmental conditions.

9. **Power Cycling Test**:
   - Cycle the ECU's power on and off during operation.
   - Verify that the ECU resumes operation correctly without issues after each cycle.

10. **Load Simulation Test**:
    - Simulate real-world driving scenarios with varying loads (e.g., city driving, highway driving).
    - Measure the ECU's power consumption and torque response to ensure efficiency and reliability.

</test_sceanrios>

<clarifications>
- Clarify if the ECU is designed to automatically detect and adapt to different voltage systems or if it requires 
manual configuration.
- Define specific metrics or thresholds for "efficient power consumption" under different load conditions.
- Specify the exact load conditions or scenarios that need to be simulated for testing.
- Confirm if there are any safety mechanisms or failure modes that need to be tested beyond basic operation.
- Define the environmental conditions and temperature ranges the ECU is expected to operate within.
</clarifications>

Clarification:  - Clarify if the ECU is designed to automatically detect and adapt to different voltage systems or 
if it requires manual configuration.
- Define specific metrics or thresholds for "efficient power consumption" under different load conditions.
- Specify the exact load conditions or scenarios that need to be simulated for testing.
- Confirm if there are any safety mechanisms or failure modes that need to be tested beyond basic operation.
- Define the environmental conditions and temperature ranges the ECU is expected to operate within.

<re.Match object; span=(2507, 3081), match='<clarifications>\n- Clarify if the ECU is designe>


...

Please clarify the questions:

 No need to test any other DTC other than DTC_OverCurrent and DTC_Voltage_Failure 


Waiting for 40 sec

Agent generating the response again

APIStatusError: Error code: 413 - {'error': {'message': 'Request too large for model `deepseek-r1-distill-llama-70b` in organization `org_01jcw5kw32e2btxg9qkcncq2tt` service tier `on_demand` on tokens per minute (TPM): Limit 6000, Requested 12408, please reduce your message size and try again. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}

In [88]:
import time

print("Start")
time.sleep(60)  # Waits for 5 seconds
print("End after 5 seconds delay")

Start
End after 5 seconds delay
